# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [3]:
df1 = df1.drop_duplicates()

In [4]:
df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

In [5]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance',  '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [6]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [7]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [8]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [9]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [10]:
from datetime import timedelta, datetime

In [11]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], format='%d/%m/%Y')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

In [12]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [13]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])
df1['screening_days'] = df1['screening_days'].astype(int)

In [14]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [15]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [16]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [17]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,2023-11-17,31,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,2023-11-17,11,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2023-11-17,18,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,2023-11-17,10,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...
105,7,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,88255.52,10743,2023-11-26,10,2023-11-25,Saturday,8.22
106,8,The Old Oak,2023-11-16,GBR,Lucky Red Distrib.,67264.79,9745,2023-11-26,10,2023-11-25,Saturday,6.90
107,9,Comandante,2023-10-31,ITA,01 Distribution,49398.52,7160,2023-11-26,26,2023-11-25,Saturday,6.90
108,10,Mary E Lo Spirito Di Mezzanotte,2023-11-23,ITA,Bim Distrib. S.R.L.,33142.31,5042,2023-11-26,3,2023-11-25,Saturday,6.57


---

## Export as .csv

In [18]:
import os

In [19]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [20]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
pd.set_option('display.max_rows', 8)

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 108
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            100 non-null    int32         
 1   title                 100 non-null    object        
 2   first_screening_date  100 non-null    object        
 3   nation                100 non-null    object        
 4   distribution          100 non-null    object        
 5   daily_takings         100 non-null    float64       
 6   daily_attendance      100 non-null    int32         
 7   date_pulled           100 non-null    object        
 8   screening_days        100 non-null    int32         
 9   date                  100 non-null    datetime64[ns]
 10  day_of_week           100 non-null    object        
 11  avg_ticket_price      100 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int32(3), object(6)
memory usage: 9.0+ KB


In [24]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,screening_days,date,avg_ticket_price
count,100.00,100.00,100.00,100.00,100,100.00
mean,5.50,178733.87,25140.49,18.96,2023-11-20 12:00:00,6.85
min,1.00,8904.68,1458.00,3.00,2023-11-16 00:00:00,4.22
25%,3.00,25299.20,4310.25,10.00,2023-11-18 00:00:00,6.29
50%,5.50,48106.60,7251.00,17.00,2023-11-20 12:00:00,6.79
75%,8.00,195602.41,28030.50,26.00,2023-11-23 00:00:00,7.36
max,10.00,1544231.00,211764.00,80.00,2023-11-25 00:00:00,10.94
std,2.89,286611.67,39947.67,15.58,NaN,1.08


In [25]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    0
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 12, dtype: int64

In [26]:
df1.nunique()

daily_rank               10
title                    17
first_screening_date     11
nation                    5
                       ... 
screening_days           11
date                     10
day_of_week               7
avg_ticket_price        100
Length: 12, dtype: int64

In [27]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.66,-0.66,0.17,-0.32
daily_takings,-0.66,1.00,0.99,0.05,0.20
daily_attendance,-0.66,0.99,1.00,0.07,0.15
screening_days,0.17,0.05,0.07,1.00,-0.47
avg_ticket_price,-0.32,0.20,0.15,-0.47,1.00


In [28]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'daily_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,screening_days,avg_ticket_price
title,,,,,
C'E' Ancora Domani,1.20,782005.15,115089.80,31.00,6.68
Napoleon,1.33,674360.93,88449.00,3.00,7.53
Hunger Games - La Ballata Dell'Usignolo E Del Serpente,2.30,341336.40,44881.70,11.00,7.44
Cento Domeniche,4.00,106978.61,15845.67,3.00,6.56
...,...,...,...,...,...
Io Capitano,8.50,15849.42,3666.00,80.00,4.34
Dream Scenario - Hai Mai Sognato Quest'Uomo?,9.43,24143.02,3622.86,10.00,6.47
Killers Of The Flower Moon,9.67,23930.22,3229.33,38.00,7.20
La Chimera,10.00,12522.45,1942.50,3.00,6.38


In [29]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'daily_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,screening_days,avg_ticket_price
title,,,,,
C'E' Ancora Domani,1.20,782005.15,115089.80,31.00,6.68
Napoleon,1.33,674360.93,88449.00,3.00,7.53
Hunger Games - La Ballata Dell'Usignolo E Del Serpente,2.30,341336.40,44881.70,11.00,7.44
Cento Domeniche,4.00,106978.61,15845.67,3.00,6.56
...,...,...,...,...,...
Io Capitano,8.50,15849.42,3666.00,80.00,4.34
Dream Scenario - Hai Mai Sognato Quest'Uomo?,9.43,24143.02,3622.86,10.00,6.47
Killers Of The Flower Moon,9.67,23930.22,3229.33,38.00,7.20
La Chimera,10.00,12522.45,1942.50,3.00,6.38
